# Recommender System Project (User-Based Collaborative Filtering)

## Imports

In [132]:
import pandas as pd
import numpy as np

## Reading Data Item Data

In [133]:
items_df = pd.read_csv("items.csv")

print(items_df.shape)
items_df.head(3)

(720, 10)


,asin,brand,title,url,image,rating,reviewUrl,totalReviews,price,originalPrice
0,B0000SX2UC,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.00,0.0
1,B0009N5L7K,Motorola,Motorola I265 phone,https://www.amazon.com/Motorola-i265-I265-phon...,https://m.media-amazon.com/images/I/419WBAVDAR...,3.0,https://www.amazon.com/product-reviews/B0009N5L7K,7,49.95,0.0
2,B000SKTZ0S,Motorola,MOTOROLA C168i AT&T CINGULAR PREPAID GOPHONE C...,https://www.amazon.com/MOTOROLA-C168i-CINGULAR...,https://m.media-amazon.com/images/I/71b+q3ydkI...,2.7,https://www.amazon.com/product-reviews/B000SKTZ0S,22,99.99,0.0


## Preprocessing Item Data

In [134]:
items_df = items_df[["asin", "rating"]]

items_df.head(3)

,asin,rating
0,B0000SX2UC,3.0
1,B0009N5L7K,3.0
2,B000SKTZ0S,2.7


## Reading Review Data

In [135]:
reviews_df = pd.read_csv("reviews.csv")

print(reviews_df.shape)
reviews_df.head(3)

(67986, 8)


,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0


## Preprocessing Review Data

In [136]:
reviews_df = reviews_df[["asin", "name", "rating"]]
reviews_df = reviews_df.groupby(['name', 'asin'])['rating'].mean().reset_index() # When a user scores a product multiple times, only save their means

reviews_df.head(3)

,name,asin,rating
0,"""I am"" Bradley",B00HPP3QD6,4.0
1,"""MASTERPIECE"" AND PEACE SHALL BE THE REWARD!",B07H8Q3C9T,5.0
2,"""monarca5""",B01N4E0RF1,5.0


## User Data

In [137]:
userInput = [
    {"asin": "B00HPP3QD6", "rating": 3.5},
    {"asin": "B07H8Q3C9T", "rating": 4.8},
    {"asin": "B01N4E0RF1", "rating": 2.1},
    {"asin": "B00MWI4KKE", "rating": 4.2},
    {"asin": "B00NKR9MJA", "rating": 3.9},
    {"asin": "B0096QYH80", "rating": 5.0},
    {"asin": "B07P8MQHSH", "rating": 2.8}
]

input_df = pd.DataFrame(userInput)
input_df.head()

,asin,rating
0,B00HPP3QD6,3.5
1,B07H8Q3C9T,4.8
2,B01N4E0RF1,2.1
3,B00MWI4KKE,4.2
4,B00NKR9MJA,3.9


## Finding Similar Users Bought the Same Products

In [138]:
reviews_subset = reviews_df[reviews_df.asin.isin(input_df.asin.to_list())]
reviews_subset = reviews_subset.dropna(subset=["asin", "name", "rating"])

print(reviews_subset.shape)

(2495, 3)


## Grouping Reviews Subset by User Names

In [139]:
grouped_reviews_subset = reviews_subset.groupby(("name"))

# grouped_reviews_subset.head()

sample_user_name = "Amazon Customer"
grouped_reviews_subset.get_group(sample_user_name)

,name,asin,rating
2539,Amazon Customer,B0096QYH80,3.031250
2562,Amazon Customer,B00HPP3QD6,3.800000
2573,Amazon Customer,B00MWI4KKE,4.065217
2576,Amazon Customer,B00NKR9MJA,3.000000
2674,Amazon Customer,B01N4E0RF1,3.254902
2873,Amazon Customer,B07H8Q3C9T,4.266667
2942,Amazon Customer,B07P8MQHSH,4.250000


## Sorting The Groups (Users With The Same Bought Products Will Be at Top)

In [140]:
sorted_grouped_reviews_subset = sorted(grouped_reviews_subset, key=(lambda g: len(g[1])), reverse=True)
sorted_grouped_reviews_subset = sorted_grouped_reviews_subset[:100]

sorted_grouped_reviews_subset[0]

('Amazon Customer',
                  name        asin    rating
 2539  Amazon Customer  B0096QYH80  3.031250
 2562  Amazon Customer  B00HPP3QD6  3.800000
 2573  Amazon Customer  B00MWI4KKE  4.065217
 2576  Amazon Customer  B00NKR9MJA  3.000000
 2674  Amazon Customer  B01N4E0RF1  3.254902
 2873  Amazon Customer  B07H8Q3C9T  4.266667
 2942  Amazon Customer  B07P8MQHSH  4.250000)

## Calculating Pearson Correlation

In [141]:
from math import sqrt

pearson_corelation = {}
input_df = input_df.sort_values(by="asin")

for username, data_df in sorted_grouped_reviews_subset:
    data_df = data_df.sort_values(by="asin")

    input_same_products = input_df[input_df.asin.isin(data_df.asin.to_list())].sort_values(by="asin")
    input_same_products_count = len(input_same_products)

    if input_same_products_count < 2: continue

    data_ratings = data_df.rating.to_list()
    input_same_products_ratings = input_same_products.rating.to_list()

    # Computing Pearson
    Sxy = sum([i*j for i, j in zip(data_ratings, input_same_products_ratings)]) - (sum(data_ratings)*sum(input_same_products_ratings))/float(input_same_products_count)
    Sxx = sum([r**2 for r in input_same_products_ratings]) - (sum(input_same_products_ratings)**2)/float(input_same_products_count)
    Syy = sum([r**2 for r in data_ratings]) - (sum(data_ratings)**2)/float(input_same_products_count)

    if (Sxx != 0 and Syy != 0):
        pearson_corelation[username] = Sxy / sqrt(Sxx * Syy)
    else:
        pearson_corelation[username] = 0

    # pearson_corelation[username] = np.corrcoef(data_ratings, input_same_products_ratings)[0]
    
pearson_corelation.values()

dict_values([0.001977010363319115, 0.10601950080823827, -0.6546536707079765, 0, 0, -0.6025490242570914, 0.9773555548504415, -0.8846153846153815, 0, 0.9999999999999979, 0, -1.0000000000000004, 0, 1.0000000000000013, 1.0000000000000004, 0.9999999999999902, 0, -1.0000000000000038, 0, -1.0, 0, -0.9999999999999992, -0.9999999999999849, 0, -1.0, 0, 0, -1.0000000000000093, -0.9999999999999979, 1.000000000000001, 0.9999999999999973, 0, 1.000000000000001, 0.9999999999999979, -1.0000000000000056, 0, 0, 0, 0, 1.0000000000000042, 0.9999999999999998, -1.0, -1.0000000000000093, 0.9999999999999913, 0, 0, 0, 1.0000000000000042, 0, 0, -0.999999999999997, 1.0000000000000042, 0.9999999999999996, 0.9999999999999986, 1.0000000000002736, 0.9999999999999992, 1.0000000000000004, 0, 0, -1.0000000000000038])

## Creating Correlation DF

In [142]:
pearson_df = pd.DataFrame.from_dict(pearson_corelation, orient="index")
pearson_df.columns = ["similarity"]
pearson_df["username"] = pearson_df.index
pearson_df.index = range(len(pearson_df))

pearson_df.head()

,similarity,username
0,0.001977,Amazon Customer
1,0.106020,Kindle Customer
2,-0.654654,Chris
3,0.000000,Eric
4,0.000000,John


## Sorting Pearson DF

In [143]:
sorted_pearson_df = pearson_df.sort_values("similarity", ascending=False)

sorted_pearson_df.head()

,similarity,username
54,1.0,carlos
47,1.0,Ron
39,1.0,Mark Twain
51,1.0,Shawn
13,1.0,Amazon customer


## Merging the Ratings of Selected Users to All Ratings

In [144]:
users_ratings_df = sorted_pearson_df.merge(reviews_df, left_on="username", right_on="name", how="inner")
users_ratings_df = users_ratings_df.drop("name", axis=1)

users_ratings_df

,similarity,username,asin,rating
0,1.0,carlos,B00836Y6B2,5.0
1,1.0,carlos,B0096QYH80,5.0
2,1.0,carlos,B00JEHJMG8,5.0
3,1.0,carlos,B01LEL8ABY,1.0
4,1.0,carlos,B06XS4978K,5.0
...,...,...,...,...
1952,-1.0,Jay,B07N4M412B,2.0
1953,-1.0,Jay,B07P8MQHSH,3.0
1954,-1.0,Jay,B07PV7LN4X,2.0
1955,-1.0,Jay,B07T7MM64L,3.0


## Multiplying the Similarities by Users' Ratings

In [145]:
users_ratings_df["weighted rating"] = users_ratings_df.similarity * users_ratings_df.rating

users_ratings_df.head()

,similarity,username,asin,rating,weighted rating
0,1.0,carlos,B00836Y6B2,5.0,5.0
1,1.0,carlos,B0096QYH80,5.0,5.0
2,1.0,carlos,B00JEHJMG8,5.0,5.0
3,1.0,carlos,B01LEL8ABY,1.0,1.0
4,1.0,carlos,B06XS4978K,5.0,5.0


## Applying Sum to Users_Ratings_DF After Grouping it by Asin

In [146]:
sum_of_users_ratings_df = users_ratings_df.groupby("asin").sum()
sum_of_users_ratings_df = sum_of_users_ratings_df[["similarity", "weighted rating"]]
sum_of_users_ratings_df.columns = ["sum of similarities", "sum of weighted ratings"]

sum_of_users_ratings_df

,sum of similarities,sum of weighted ratings
asin,,
B0000SX2UC,0.001977,0.005931
B000SKTZ0S,0.001977,0.001977
B001AO4OUC,0.001977,0.009885
B001DCJAJG,0.106020,0.424078
B002AS9WEA,0.001977,0.004943
...,...,...
B07Z8BL2VW,0.001977,0.009885
B07ZDJCL76,0.001977,0.007249
B07ZHPCJW3,0.000000,0.000000


## Recommendation DF

In [147]:
recommendation_df = pd.DataFrame()
recommendation_df["Average Weighted Rating"] = sum_of_users_ratings_df["sum of similarities"] / sum_of_users_ratings_df["sum of weighted ratings"]
recommendation_df = recommendation_df.dropna(subset=["Average Weighted Rating"])

recommendation_df

,Average Weighted Rating
asin,
B0000SX2UC,0.333333
B000SKTZ0S,1.000000
B001AO4OUC,0.200000
B001DCJAJG,0.250000
B002AS9WEA,0.400000
...,...
B07Z41LHPB,1.000000
B07Z8BL2VW,0.200000
B07ZDJCL76,0.272727


## Sorting Recommendation DF

In [148]:
sorted_recommendation_df = recommendation_df.sort_values("Average Weighted Rating", ascending=False)
sorted_recommendation_df["asin"] = sorted_recommendation_df.index
sorted_recommendation_df.index = range(len(sorted_recommendation_df))

sorted_recommendation_df

,Average Weighted Rating,asin
0,13.074669,B07PV7LN4X
1,2.183441,B07PY52GVP
2,1.936983,B07H6V8TB9
3,1.927559,B00JS73V2U
4,1.924217,B0096QYH80
...,...,...
540,-0.896505,B01M0PADR4
541,-0.969552,B07QK32C8M
542,-1.142698,B06X9HVVC5
543,-2.252030,B07Q84DPZH


## Recommended Product Data

In [150]:
items_df = pd.read_csv("items.csv")

count = 5
recommended_items = items_df[items_df.asin.isin(sorted_recommendation_df.head(count).asin.to_list())]

recommended_items

,asin,brand,title,url,image,rating,reviewUrl,totalReviews,price,originalPrice
31,B0096QYH80,Motorola,"Motorola Droid RAZR M XT907 Verizon Wireless, ...",https://www.amazon.com/Motorola-Droid-RAZR-Ver...,https://m.media-amazon.com/images/I/71-afm8RuL...,3.0,https://www.amazon.com/product-reviews/B0096QYH80,488,349.99,0.0
66,B00JS73V2U,Motorola,"Motorola DROID MAXX, Black 16GB (Verizon Wirel...",https://www.amazon.com/Motorola-DROID-MAXX-Ver...,https://m.media-amazon.com/images/I/717w0Z516K...,4.1,https://www.amazon.com/product-reviews/B00JS73V2U,263,0.00,0.0
445,B07H6V8TB9,Sony,"Sony Xperia XZ3 Unlocked Smartphone, 64GB - 6....",https://www.amazon.com/Sony-Xperia-Unlocked-Sm...,https://m.media-amazon.com/images/I/81eb1nqD8l...,4.1,https://www.amazon.com/product-reviews/B07H6V8TB9,83,0.00,0.0
554,B07PV7LN4X,Samsung,Samsung Galaxy A30 Dual SIM 32GB (SM-A305G/DS)...,https://www.amazon.com/Samsung-A30-SM-A305G-Un...,https://m.media-amazon.com/images/I/71i9lcnWT1...,4.2,https://www.amazon.com/product-reviews/B07PV7LN4X,352,215.00,0.0
557,B07PY52GVP,Xiaomi,"Xiaomi Redmi Note 7, 64GB/4GB RAM, 6.30'' FHD+...",https://www.amazon.com/Xiaomi-Redmi-Note-Snapd...,https://m.media-amazon.com/images/I/61ZaskM5hB...,4.4,https://www.amazon.com/product-reviews/B07PY52GVP,630,167.00,0.0
